# CE1115 Mini Project

### Group 7
Ryu, Rayne, Jia Rui
### What We Want To Explore!!
How Does The Genre Of A Movie Affects Its Chances To Win An Award?

In [1]:
# Importing neccesary libraries 
from tmdbv3api import TMDb, Movie, Discover
from tmdbv3api import Account
from tmdbv3api import Authentication
import numpy as np
import random
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
from collections import defaultdict

In [2]:
##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47'
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()

# Oscar, Golden Globes, Bafta Datasets

In [3]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
golden_df = pd.read_csv('data/golden_globe_awards.csv')
bafta_df = pd.read_csv('data/bafta_films.csv')

## Cleaning the datasets

In [4]:
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


In [5]:
golden_df

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True
...,...,...,...,...,...,...,...
7986,2019,2020,77,Best Performance by an Actor in a Supporting R...,Kieran Culkin,Succession,False
7987,2019,2020,77,Best Performance by an Actor in a Supporting R...,Andrew Scott,Fleabag,False
7988,2019,2020,77,Best Performance by an Actor in a Supporting R...,Henry Winkler,Barry,False
7989,2019,2020,77,Cecil B. deMille Award,Tom Hanks,NaN,False


In [6]:
bafta_df

,year,category,nominee,workers,winner
0,1949,Film | British Film in 1949,The Fallen Idol,NaN,True
1,1949,Film | British Film in 1949,Hamlet,NaN,False
2,1949,Film | British Film in 1949,Oliver Twist,NaN,False
3,1949,Film | British Film in 1949,Once A Jolly Swagman,NaN,False
4,1949,Film | British Film in 1949,The Red Shoes,NaN,False
...,...,...,...,...,...
4171,2020,Film | Supporting Actress in 2020,Marriage Story,Laura Dern,True
4172,2020,Film | Supporting Actress in 2020,Jojo Rabbit,Scarlett Johansson,False
4173,2020,Film | Supporting Actress in 2020,Little Women,Florence Pugh,False
4174,2020,Film | Supporting Actress in 2020,Bombshell,Margot Robbie,False


In [7]:
oscar_df= oscar_df.rename({'category':'oscar_category', 'name':'nominee', 'winner':'oscar_win'}, axis=1)
oscar_df.drop(['year_ceremony', 'ceremony'], axis=1, inplace=True)
oscar_df = oscar_df.query('year_film >=1990')

In [8]:
oscar_df['oscar_category'].unique()

array(['ACTOR IN A LEADING ROLE', 'ACTOR IN A SUPPORTING ROLE',
       'ACTRESS IN A LEADING ROLE', 'ACTRESS IN A SUPPORTING ROLE',
       'ART DIRECTION', 'CINEMATOGRAPHY', 'COSTUME DESIGN', 'DIRECTING',
       'DOCUMENTARY (Feature)', 'DOCUMENTARY (Short Subject)',
       'FILM EDITING', 'FOREIGN LANGUAGE FILM', 'MAKEUP',
       'MUSIC (Original Score)', 'MUSIC (Original Song)', 'BEST PICTURE',
       'SHORT FILM (Animated)', 'SHORT FILM (Live Action)', 'SOUND',
       'SOUND EFFECTS EDITING',
       'WRITING (Screenplay Based on Material from Another Medium)',
       'WRITING (Screenplay Written Directly for the Screen)',
       'SPECIAL ACHIEVEMENT AWARD (Visual Effects)', 'HONORARY AWARD',
       'IRVING G. THALBERG MEMORIAL AWARD', 'VISUAL EFFECTS',
       'WRITING (Screenplay Based on Material Previously Produced or Published)',
       'JEAN HERSHOLT HUMANITARIAN AWARD',
       'MUSIC (Original Dramatic Score)',
       'MUSIC (Original Musical or Comedy Score)',
       'SPECIAL 

We want to look at a smaller pool of awards.
<br> We want to have:
<br>'ACTOR IN A LEADING ROLE',
<br>'ACTOR IN A SUPPORTING ROLE',
<br>'ACTRESS IN A LEADING ROLE', 
<br>'ACTRESS IN A SUPPORTING ROLE'
<br>'DIRECTING',
<br>'CINEMATOGRAPHY',
<br>'FILM EDITING'
<br>'MUSIC (Original Score)'
<br>'BEST PICTURE',
<br>'WRITING (Screenplay Based on Material from Another Medium)',(Adapted Screenplay)
<br>'WRITING (Screenplay Written Directly for the Screen)' (Oringal Screenplay)

In [9]:
oscar_df = oscar_df.replace(to_replace={'oscar_category' : {'ACTOR IN A LEADING ROLE': 'Best Actor',
                                                'ACTOR IN A SUPPORTING ROLE' : 'Best Supporting Actor',
                                                'ACTRESS IN A LEADING ROLE': 'Best Actress',
                                                'ACTRESS IN A SUPPORTING ROLE' : 'Best Supporting Actress',
                                                'DIRECTING' : 'Best Director',
                                                'BEST PICTURE' : 'Best Pictre',
                                                'CINEMATOGRAPHY' : 'Best Cinematography',
                                                'FILM EDITING' : 'Best Film Editing',
                                                'MUSIC (Original Score)' : 'Best Original Score',
                                                'WRITING (Adapted Screenplay)' : 'Best Adapted Screenplay',
                                                'WRITING (Original Screenplay)' : 'Best Original Screenplay',},}, value=None)

oscar_df = oscar_df.query('oscar_category in ["Best Actor", "Best Supporting Actor", "Best Actress", "Best Supporting Actress", "Best Director", "Best Pictre", "Best Cinematography", "Best Film Editing", "Best Original Score","Best Adapted Screenplay","Best Original Screenplay"]')
oscar_df = oscar_df.reset_index()

In [127]:
filmD = oscar_df[["film", "year_film", "oscar_category", "nominee"]].to_dict('r')
oscar_dict = {"movie_id":[], "film":[], "oscar_category":[], "nominee":[], "genre_id":[], "vote_average":[]};
movie = Movie()
for item in filmD:
    print(item["film"])
    s = movie.search(item["film"])
    for i in range(0,10):
        result = s[i]
        if result.release_date[0:4] == str(item["year_film"]) or result.release_date[0:4] == str(item["year_film"]-1) or result.release_date[0:4] == str(item["year_film"]+1):
            oscar_dict["movie_id"].append(result.id)
            oscar_dict["film"].append(item["film"])
            oscar_dict["oscar_category"].append(item["oscar_category"])
            oscar_dict["nominee"].append(item["nominee"])
            oscar_dict["genre_id"].append(result.genre_ids)
            oscar_dict["vote_average"].append(result.vote_average)
            break
            
details_df = pd.DataFrame(oscar_dict)

Dances With Wolves
Awakenings
Cyrano de Bergerac
The Field
Reversal of Fortune
Longtime Companion
The Godfather, Part III
Dances With Wolves
Dick Tracy
GoodFellas
Misery
The Grifters
Pretty Woman
Postcards from the Edge
Mr. & Mrs. Bridge
The Grifters
GoodFellas
Ghost
Wild at Heart
Dances With Wolves
Avalon
Dances With Wolves
Dick Tracy
The Godfather, Part III
Henry & June
Dances With Wolves
The Godfather, Part III
GoodFellas
The Grifters
Reversal of Fortune
Dances With Wolves
Ghost
The Godfather, Part III
GoodFellas
The Hunt for Red October
Avalon
Dances With Wolves
Ghost
Havana
Home Alone
Awakenings
Dances With Wolves
Ghost
The Godfather, Part III
GoodFellas
Bugsy
Cape Fear
The Silence of the Lambs
The Prince of Tides
The Fisher King
JFK
Bugsy
Bugsy
Barton Fink
City Slickers
Thelma & Louise
Rambling Rose
The Silence of the Lambs
For the Boys
Thelma & Louise
Rambling Rose
Cape Fear
The Prince of Tides
The Fisher King
Fried Green Tomatoes
Bugsy
JFK
The Prince of Tides
Terminator 2: Judg

Zero Dark Thirty
Argo
Beasts of the Southern Wild
Life of Pi
Lincoln
Silver Linings Playbook
Amour
Django Unchained
Flight
Moonrise Kingdom
Zero Dark Thirty
American Hustle
Nebraska
The Wolf of Wall Street
12 Years a Slave
Dallas Buyers Club
Captain Phillips
American Hustle
12 Years a Slave
The Wolf of Wall Street
Dallas Buyers Club
American Hustle
Blue Jasmine
Gravity
Philomena
August: Osage County
Blue Jasmine
American Hustle
12 Years a Slave
August: Osage County
Nebraska
The Grandmaster
Gravity
Inside Llewyn Davis
Nebraska
Prisoners
American Hustle
Gravity
Nebraska
12 Years a Slave
The Wolf of Wall Street
American Hustle
Captain Phillips
Dallas Buyers Club
Gravity
12 Years a Slave
The Book Thief
Gravity
Her
Philomena
Saving Mr. Banks
American Hustle
Captain Phillips
Dallas Buyers Club
Gravity
Her
Nebraska
Philomena
12 Years a Slave
The Wolf of Wall Street
Before Midnight
Captain Phillips
Philomena
12 Years a Slave
The Wolf of Wall Street
American Hustle
Blue Jasmine
Dallas Buyers Cl

In [129]:
oscar_details = pd.merge(oscar_df, details_df, on=['film', 'oscar_category', 'nominee'], how='left')
oscar_details['movie_id'] = oscar_details.movie_id.fillna(0).astype(int)

In [130]:
oscar_details

,index,year_film,oscar_category,nominee,film,oscar_win,movie_id,genre_id,vote_average
0,6889,1990,Best Actor,Kevin Costner,Dances With Wolves,False,581,"[12, 18, 37]",7.9
1,6890,1990,Best Actor,Robert De Niro,Awakenings,False,11005,[18],7.8
2,6891,1990,Best Actor,Gerard Depardieu,Cyrano de Bergerac,False,11673,"[18, 35, 36, 10749]",7.3
3,6892,1990,Best Actor,Richard Harris,The Field,False,31651,[18],6.9
4,6893,1990,Best Actor,Jeremy Irons,Reversal of Fortune,True,38718,[18],6.7
...,...,...,...,...,...,...,...,...,...
1548,10386,2019,Best Original Screenplay,Written by Rian Johnson,Knives Out,False,546554,"[35, 80, 18, 9648]",7.8
1549,10387,2019,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False,492188,[18],7.8
1550,10388,2019,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,530915,"[10752, 18, 28, 53]",7.9
1551,10389,2019,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,466272,"[35, 18, 53]",7.5


In [131]:
movie = Movie()
detail_dict = {"movie_id":[], "budget":[], "revenue":[], "release_date":[], "runtime":[], "film":[], "oscar_category":[], "nominee":[]};
get_details = oscar_details.to_dict('r')
for item in get_details:
    if item["movie_id"] != 0:
        m = movie.details(item["movie_id"])
        detail_dict["movie_id"].append(item["movie_id"])
        detail_dict["budget"].append(m.budget)
        detail_dict["revenue"].append(m.revenue)
        detail_dict["release_date"].append(m.release_date)
        detail_dict["runtime"].append(m.runtime)
        detail_dict["film"].append(item["film"])
        detail_dict["oscar_category"].append(item["oscar_category"])
        detail_dict["nominee"].append(item["nominee"])
        
more_details_df = pd.DataFrame(detail_dict)
more_details_df

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,movie_id,budget,revenue,release_date,runtime,film,oscar_category,nominee
0,581,22000000,424208848,1990-03-30,181,Dances With Wolves,Best Actor,Kevin Costner
1,11005,0,52096475,1990-12-04,120,Awakenings,Best Actor,Robert De Niro
2,11673,0,0,1990-01-01,137,Cyrano de Bergerac,Best Actor,Gerard Depardieu
3,31651,0,1494399,1990-09-21,110,The Field,Best Actor,Richard Harris
4,38718,0,15445131,1990-10-05,111,Reversal of Fortune,Best Actor,Jeremy Irons
...,...,...,...,...,...,...,...,...
1544,546554,40000000,309232797,2019-11-27,131,Knives Out,Best Original Screenplay,Written by Rian Johnson
1545,492188,19000000,2300000,2019-11-06,137,Marriage Story,Best Original Screenplay,Written by Noah Baumbach
1546,530915,100000000,374733942,2019-12-25,119,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns
1547,466272,95000000,374251247,2019-07-24,162,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino


In [132]:
oscar_details = pd.merge(oscar_details, more_details_df, on=['movie_id', 'film', 'oscar_category', 'nominee'],  how='left')
oscar_details

,index,year_film,oscar_category,nominee,film,oscar_win,movie_id,genre_id,vote_average,budget,revenue,release_date,runtime
0,6889,1990,Best Actor,Kevin Costner,Dances With Wolves,False,581,"[12, 18, 37]",7.9,22000000.0,424208848.0,1990-03-30,181.0
1,6890,1990,Best Actor,Robert De Niro,Awakenings,False,11005,[18],7.8,0.0,52096475.0,1990-12-04,120.0
2,6891,1990,Best Actor,Gerard Depardieu,Cyrano de Bergerac,False,11673,"[18, 35, 36, 10749]",7.3,0.0,0.0,1990-01-01,137.0
3,6892,1990,Best Actor,Richard Harris,The Field,False,31651,[18],6.9,0.0,1494399.0,1990-09-21,110.0
4,6893,1990,Best Actor,Jeremy Irons,Reversal of Fortune,True,38718,[18],6.7,0.0,15445131.0,1990-10-05,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,10386,2019,Best Original Screenplay,Written by Rian Johnson,Knives Out,False,546554,"[35, 80, 18, 9648]",7.8,40000000.0,309232797.0,2019-11-27,131.0
1549,10387,2019,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False,492188,[18],7.8,19000000.0,2300000.0,2019-11-06,137.0
1550,10388,2019,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,530915,"[10752, 18, 28, 53]",7.9,100000000.0,374733942.0,2019-12-25,119.0
1551,10389,2019,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,466272,"[35, 18, 53]",7.5,95000000.0,374251247.0,2019-07-24,162.0


In [159]:
#oscar_details.drop(['index'], axis=1, inplace=True)
titles = list(oscar_details.columns)
titles[0], titles[1] = titles[1], titles[0]
titles[1], titles[3] = titles[3], titles[1]
titles[4], titles[5] = titles[5], titles[4]
titles

['movie_id',
 'film',
 'nominee',
 'year_film',
 'oscar_category',
 'oscar_win',
 'genre_id',
 'vote_average',
 'budget',
 'revenue',
 'release_date',
 'runtime']

In [160]:
oscar_details = oscar_details[titles]
oscar_details

,movie_id,film,nominee,year_film,oscar_category,oscar_win,genre_id,vote_average,budget,revenue,release_date,runtime
0,581,Dances With Wolves,Kevin Costner,1990,Best Actor,False,"[12, 18, 37]",7.9,22000000.0,424208848.0,1990-03-30,181.0
1,11005,Awakenings,Robert De Niro,1990,Best Actor,False,[18],7.8,0.0,52096475.0,1990-12-04,120.0
2,11673,Cyrano de Bergerac,Gerard Depardieu,1990,Best Actor,False,"[18, 35, 36, 10749]",7.3,0.0,0.0,1990-01-01,137.0
3,31651,The Field,Richard Harris,1990,Best Actor,False,[18],6.9,0.0,1494399.0,1990-09-21,110.0
4,38718,Reversal of Fortune,Jeremy Irons,1990,Best Actor,True,[18],6.7,0.0,15445131.0,1990-10-05,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1548,546554,Knives Out,Written by Rian Johnson,2019,Best Original Screenplay,False,"[35, 80, 18, 9648]",7.8,40000000.0,309232797.0,2019-11-27,131.0
1549,492188,Marriage Story,Written by Noah Baumbach,2019,Best Original Screenplay,False,[18],7.8,19000000.0,2300000.0,2019-11-06,137.0
1550,530915,1917,Written by Sam Mendes & Krysty Wilson-Cairns,2019,Best Original Screenplay,False,"[10752, 18, 28, 53]",7.9,100000000.0,374733942.0,2019-12-25,119.0
1551,466272,Once upon a Time...in Hollywood,Written by Quentin Tarantino,2019,Best Original Screenplay,False,"[35, 18, 53]",7.5,95000000.0,374251247.0,2019-07-24,162.0
